In [1]:
import os, json, re    

In [2]:
def importJSONFiles(files: list, path: str) -> list[dict[str, any]]:

    hts_data = []

    for file in files:

        with open(f'{path}/{file}') as f:
            header = re.sub(r'\.json', '', file)
            hts_data.append(
                {
                    'header': header,
                    'data': json.loads(f.read())
                })

    return hts_data

def getHTSRecord(hts_data: list[dict[str, any]], queryStr: str):

    for record in hts_data:
        
        if record['header'] == queryStr:
            return record['data']
    
    return 'No record found'

In [3]:
def checkResultQuery(result_query: list[dict[str, any]]):

    index = 0
    indent_list = [record['indent'] for record in result_query if 'indent' in record]
    
    for i in range(0, result_query[-1]['indent']):
        
        if i not in indent_list:
    
            if 'missing' in result_query[index-1]:

                result_query[index-1]['missing'].append(i)
            else:
                result_query[index-1]['missing'] = [i]
                result_query[index-1]['recordIndex'] = index-1

        else:
            index += 1



In [4]:
def grabQueryRecords(hts_record: list[dict[str, any]], query: list[str]) -> list[dict[str, any]]:

    result = []
    index_query = 0

    while index_query < len(query):

        for record in hts_record:

            if 'htsno' in record and re.match(rf'{query[index_query]}$', record['htsno']):

                result.append({
                    'htsno': record['htsno'],
                    'indent': record['indent'],
                    'description': record['description']
                })

        index_query += 1
    
    checkResultQuery(result)
    
    return result

In [5]:
path = '../../db_hts/temp/NEW_final_json_files'
query = ["0201", "0201.20", "0201.20.06", "0201.20.06.00"]
files = os.listdir(path)
hts_data = importJSONFiles(files, path)

In [6]:
hts_record = getHTSRecord(hts_data, query[0])

In [7]:
result = grabQueryRecords(hts_record, query)
result

[{'htsno': '0201',
  'indent': 0,
  'description': 'Meat of bovine animals, fresh or chilled:'},
 {'htsno': '0201.20',
  'indent': 1,
  'description': 'Other cuts with bone in:',
  'missing': [2],
  'recordIndex': 1},
 {'htsno': '0201.20.06.00', 'indent': 3, 'description': 'Other'}]